In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from Methods.costs import * 
from Methods.least_squares import * 
from Methods.ridge import *
from Methods.cross_validation import *
from Methods.split_data import *
from Methods.scaling_standardization import *
from Methods.build_polynomial import *
from Methods.clearDataset import *

%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from Methods.proj1_helpers import *
DATA_TRAIN_PATH = 'csv/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
tX_tra = tX.T

## Do your thing crazy machine learning thing here :) ...

# Raw Data Analysis 

### Visualisation of the data

In [3]:
#Boxplot of the Raw Data
#boxplot(tX, -2000, 6000, 'boxplot_original')

# Feature Processing

## Feature Deletion

In [4]:
#PERCENT_FACTOR = 0.5
#dele_tX_temp = deleteNoneWantedData(tX, PERCENT_FACTOR)
#dele_tX = deleteUnwantedLine(dele_tX_temp)

## Replace Undefined variables by mean of all the other variables

In [5]:
tX = np.array(averageData(tX))

## Data scaling 

In [6]:
tX = data_scaling(tX.T)

##### Boxplot of the scale data

In [7]:
#boxplot(tX, -0.5, 1.5, 'boxplot_scaled')

## Data standardization

In [8]:
#tX = data_standardization(tX.T)

##### Boxplot of the Standardized data

In [9]:
#boxplot(std_tX, -7, 7, 'boxplot_standardized')

## Big Magic Method

In [10]:
#for i in range(0, 30):
#print(tX.shape)
tX = add_feature(y, tX.T[1], tX, multiply,degree=2)

#print(tX.shape)
#newx = add_feature(y, , tX ,multiply,degree=2)  

a = np.array([1, 2, 3])

b = multiply(a, degree=2)
c = log_def(a)

print('tX.T[1]')
print(tX.T[1])

print('lol')

tX = add_feature(y, tX.T[2], tX, multiply, degree=2)


print('tX.T[1]')
print(tX.T[1])


x.shape
(250000,)
y.shape
(250000,)
tX.shape
(250000, 30)
tX.T[1]
[ 0.01396362  0.01739099  0.03609781 ...,  0.01574029  0.00749603
  0.0181897 ]
lol
x.shape
(250000,)
y.shape
(250000,)
tX.shape
(250000, 30)
tX.T[1]
[ 0.03585047  0.01739099  0.02429399  0.01326085  0.00409965  0.07807848
  0.00345367  0.0043138   0.00403441  0.0286845   0.00379428  0.00390143
  0.00371002  0.01203274  0.0040266   0.00301519  0.01101212  0.00371857
  0.00363886  0.01257149  0.0032345   0.03657327  0.00381851  0.01287631
  0.00376343  0.00385016  0.0151702   0.00361586  0.00361792  0.01287631
  0.0005902 ]


### Cross Validation

### Least Squares

In [ ]:
seed = 1
k_fold = 10

# split data in k fold
k_indices = build_k_indices(y, k_fold, seed)

# define lists to store the loss of training data and test data
rmse_tr = []
rmse_te = []
k_list = list(range(k_fold))
tot_loss_tr = 0
tot_loss_te = 0
for k in k_list:
    loss_tr, loss_te, accuracy_least = cross_validation(y, tX, k_indices, k, least_squares)
    tot_loss_tr += loss_tr
    tot_loss_te += loss_te
rmse_tr.append(np.sqrt(2/k_fold * tot_loss_tr))
rmse_te.append(np.sqrt(2/k_fold * tot_loss_te))

accuracy_least

### Ridge

In [ ]:
seed = 1
k_fold = 10
lambdas = np.logspace(-5, 1, 20)

# split data in k fold
k_indices = build_k_indices(y, k_fold, seed)

# define lists to store the loss of training data and test data
rmse_tr = []
rmse_te = []
best_accuracy = 0
best_lamb = 0
best_k = 0
k_list = list(range(k_fold))
for lamb in lambdas:
    tot_loss_tr = 0
    tot_loss_te = 0
    for k in k_list:
        loss_tr, loss_te, accuracy_ridge = cross_validation(y, tX, k_indices, k, ridge_regression, lamb = lamb)
        if accuracy_ridge > best_accuracy:
            best_accuracy = accuracy_ridge
            best_lamb = lamb
            best_k = k
        tot_loss_tr += loss_tr
        tot_loss_te += loss_te
    rmse_tr.append(np.sqrt(2/k_fold * tot_loss_tr))
    rmse_te.append(np.sqrt(2/k_fold * tot_loss_te))
#plt.boxplot(rmse_te)
#plt.show()
#print(rmse_te)
#cross_validation_visualization(lambdas, rmse_tr, rmse_te)
print(best_accuracy)
print(best_k)
print(best_lamb)


###  Bias-Variance decomposition

### Binary Classification

In [ ]:
#Use predict_labels de project helpers (fait la multiplication entre tX et les w)

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = 'csv/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = 'csv/sample-submission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)